In [1]:
import boto3
import os
from concurrent import futures
import pickle
from glob import glob

In [2]:
with open('list_img_urls.data', 'rb') as filehandle:
    urls = pickle.load(filehandle)

In [3]:
len_root = len('https://s3-ap-southeast-2.amazonaws.com/imos-data/')

In [4]:
keys = []
for url in urls:
    keys.append(url[len_root:])
keys = list(set(keys))

In [16]:
already_downloaded = glob('/Users/diegoeduardo/Desktop/Coral Reef Data/images/*.tif')
names = []
for path in already_downloaded:
    names.append(path.split('/')[-1])

In [6]:
len(names) == len(already_downloaded)

True

In [17]:
new_keys = []
for file in keys:
    name = file.split('/')[-1]
    if (name not in names) and (name not in error_keys):
        new_keys.append(file)

In [18]:
len(new_keys) == len(keys) - len(already_downloaded) - len(error_keys)

False

In [9]:
new_keys = new_keys[::-1]
error_keys = []

In [19]:
CONCURRENT_DOWNLOADS = 20
BUCKET_NAME = 'imos-data'
DEST_DIR = '/Users/diegoeduardo/Desktop/Coral Reef Data/images/'

os.makedirs(DEST_DIR, exist_ok=True)

S3_OBJECT_KEYS = new_keys # List of S3 object keys


def fetch(key):
    s3 = boto3.resource('s3')
    file_path = DEST_DIR + key.split('/')[-1]
    s3.Bucket(BUCKET_NAME).download_file(key, file_path)
    return file_path


def fetch_all(keys):

    with futures.ThreadPoolExecutor(max_workers=CONCURRENT_DOWNLOADS) as executor:
        future_to_key = {executor.submit(fetch, key): key for key in keys}
        print("All URLs submitted.")
        for future in futures.as_completed(future_to_key):
            key = future_to_key[future]
            if future.exception() is None:
                yield key, future.result()
            else:
                error_keys.append(key)
                yield key, future.exception()


for key, file_loc in fetch_all(S3_OBJECT_KEYS):
    print('key: {}\npath: {}'.format(key, file_loc))

All URLs submitted.
key: IMOS/AUV/WA201104/r20110410_011202_rottnest_07_25m_s_out/i20110410_011202_gtif/PR_20110415_223857_326_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110410_011202_rottnest_07_25m_s_out/i20110410_011202_gtif/PR_20110415_223857_326_LC16.tif"
key: IMOS/AUV/Batemans201011/r20101119_224028_site6guz_11_densegrid3/i20101119_224028_gtif/PR_20101218_220953_429_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/Batemans201011/r20101119_224028_site6guz_11_densegrid3/i20101119_224028_gtif/PR_20101218_220953_429_LC16.tif"
key: IMOS/AUV/Batemans201011/r20101119_224028_site6guz_11_densegrid3/i20101119_224028_gtif/PR_20101217_040042_867_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/Batemans201011/r20101119_224028_site6guz_11_densegrid3/i20101119_224028_gtif/PR_20101217_040042_867_LC16.tif"
key: IMOS/AUV/WA201304/r2

key: IMOS/AUV/Batemans201011/r20101119_224028_site6guz_11_densegrid3/i20101119_224028_gtif/PR_20101218_222753_507_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/Batemans201011/r20101119_224028_site6guz_11_densegrid3/i20101119_224028_gtif/PR_20101218_222753_507_LC16.tif"
key: IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101216_014945_247_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101216_014945_247_LC16.tif"
key: IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101218_212346_172_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101218_212346_172_LC16.tif"
key: IMOS/AUV/Tasmania200810/r20081008_224114_hippo

key: IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/PR_20110416_060154_537_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/PR_20110416_060154_537_LC16.tif"
key: IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101216_020632_307_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101216_020632_307_LC16.tif"
key: IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/PR_20110416_052851_411_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/PR_20110416_052851_411_LC16.tif"
key: IMOS/AUV/WA201104/r20110409_060541_rottnest_06_15m_n_out/i20110409_060541_gt

key: IMOS/AUV/WA201104/r20110409_060541_rottnest_06_15m_n_out/i20110409_060541_gtif/PR_20110415_221237_228_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110409_060541_rottnest_06_15m_n_out/i20110409_060541_gtif/PR_20110415_221237_228_LC16.tif"
key: IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101216_021623_397_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101216_021623_397_LC16.tif"
key: IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_230604_133_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_230604_133_LC16.tif"
key: IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_

key: IMOS/AUV/WA201104/r20110410_011202_rottnest_07_25m_s_out/i20110410_011202_gtif/PR_20110415_233304_287_LC16.tif
path: An error occurred (404) when calling the HeadObject operation: Not Found
key: IMOS/AUV/WA201104/r20110409_060541_rottnest_06_15m_n_out/i20110409_060541_gtif/PR_20110415_221637_247_LC16.tif
path: An error occurred (404) when calling the HeadObject operation: Not Found
key: IMOS/AUV/Batemans201011/r20101119_224028_site6guz_11_densegrid3/i20101119_224028_gtif/PR_20101218_212933_208_LC16.tif
path: An error occurred (404) when calling the HeadObject operation: Not Found
key: IMOS/AUV/Tasmania200810/r20081007_224547_ohara_07_transect/i20081007_224547_gtif/PR_20081008_002946_833_LC16.tif
path: /Users/diegoeduardo/Desktop/Coral Reef Data/images/PR_20081008_002946_833_LC16.tif
key: IMOS/AUV/WA201104/r20110412_025750_jurien_14_48m_n/i20110412_025750_gtif/PR_20110412_033514_529_LC16.tif
path: /Users/diegoeduardo/Desktop/Coral Reef Data/images/PR_20110412_033514_529_LC16.tif
ke

...

key: IMOS/AUV/WA201104/r20110409_060541_rottnest_06_15m_n_out/i20110409_060541_gtif/PR_20110415_223837_351_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110409_060541_rottnest_06_15m_n_out/i20110409_060541_gtif/PR_20110415_223837_351_LC16.tif"
key: IMOS/AUV/WA201104/r20110410_011202_rottnest_07_25m_s_out/i20110410_011202_gtif/PR_20110415_223037_314_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110410_011202_rottnest_07_25m_s_out/i20110410_011202_gtif/PR_20110415_223037_314_LC16.tif"
key: IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101217_023106_142_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101217_023106_142_LC16.tif"
key: IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_

key: IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/PR_20110416_052231_356_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/PR_20110416_052231_356_LC16.tif"
key: IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_232404_218_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_232404_218_LC16.tif"
key: IMOS/AUV/WA201104/r20110409_060541_rottnest_06_15m_n_out/i20110409_060541_gtif/PR_20110415_222437_285_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110409_060541_rottnest_06_15m_n_out/i20110409_060541_gtif/PR_20110415_222437_285_LC16.tif"
key: IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/

key: IMOS/AUV/WA201104/r20110412_073109_jurien_15_25m_n/i20110412_073109_gtif/PR_20110415_232624_256_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110412_073109_jurien_15_25m_n/i20110412_073109_gtif/PR_20110415_232624_256_LC16.tif"
key: IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_231424_199_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_231424_199_LC16.tif"
key: IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/PR_20110416_053251_430_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/PR_20110416_053251_430_LC16.tif"
key: IMOS/AUV/Batemans201011/r20101119_224028_site6guz_11_densegrid3/i20101119_224028_gtif/PR

key: IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101217_022840_124_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/PS201012/r20101219_032323_broughton_12_dense_bisz/i20101219_032323_gtif/PR_20101217_022840_124_LC16.tif"
key: IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_232204_208_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_232204_208_LC16.tif"
key: IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_230824_171_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_230824_171_LC16.tif"
key: IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_0236

key: IMOS/AUV/WA201104/r20110410_011202_rottnest_07_25m_s_out/i20110410_011202_gtif/PR_20110415_231244_164_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110410_011202_rottnest_07_25m_s_out/i20110410_011202_gtif/PR_20110415_231244_164_LC16.tif"
key: IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_222617_266_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201104/r20110410_043205_rottnest_08_25m_n_in/i20110410_043205_gtif/PR_20110415_222617_266_LC16.tif"
key: IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/PR_20110416_051431_317_LC16.tif
path: Could not connect to the endpoint URL: "https://imos-data.s3.amazonaws.com/IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/PR_20110416_051431_317_LC16.tif"
key: IMOS/AUV/WA201304/r20130406_023610_geebank_03_15m_out/i20130406_023610_gtif/

In [11]:
len(error_keys)

895

In [20]:
len(keys)

9941